In [1]:
"""
Created on Wed Aug  4 17:50:06 2021

@author: amol
"""

# set TF GPU memory growth so that it doesn't hog everything at once
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import os
import numpy as np
from main import *
debug_mode = True  # faster experiments for debugging

# util for getting objects' fields' names
field_names = lambda x: list(vars(x).keys())

In [2]:
#Prepare the DataFrame that will be used downstream
dp = DataPreparer()
dp.createPCAs()
dp.sparsePCAs()
dp.zmixOrthogonalPCAs()
df = dp.getDataframe()

# currently passing dp eventually we want to abstract all the constants into 1 class
dm = DataManager(df, dp)

In [ ]:
"""
bestModel, experimentSettings = exprExec.modelFactory.openBestModel()
dm.createTrainTestData(experimentSettings['dataSetMethod'], experimentSettings['noOfCpv'],
                       experimentSettings['ipscaler'], experimentSettings['opscaler'])
dm.input_data_cols
"""

In [6]:
'''
Run the Model Experiments
'''
model = 'PCDNN_V2'
assert model in ['PCDNN_V2', 'PCDNN_V1', 'SIMPLE_DNN', 'GP'] # valid possible models
exprExec=run_model_experiments(dm, models=model, debug_mode=debug_mode)

/home/dwyerdei/rom_project2/src/experiment_executor/pcdnn_v2_experiment_executor.py:48: UserWarning: manually overriding n models to 1! change this!
  warnings.warn('manually overriding n models to 1! change this!')


Parent DNNModelFactory Instantiated
Parent DNNModelFactory Instantiated
=================== randomequaltraintestsplit ===================
------------------ AllSpeciesAndZmix ------------------
--------------------self.build_and_compile_pcdnn_v2_model----------------------
53 4 Y Y Y Y
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 species_input (InputLayer)     [(None, 53)]         0           []                               
                                                                                                  
 zmix (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 linear_embedding (Functional)  (None, 4)            216         ['species_input[0][0]']          
   

129/129 [==============================] - 2s 14ms/step - loss: 0.0115 - mae: 0.0088 - mse: 3.8016e-04 - log_mse: -8.1227 - log_mae: -4.7701 - exp_mse_mag: -3.2023 - exp_mae_mag: -1.9625 - exp_R2: 0.9689 - R2: 0.9728 - val_loss: 0.0094 - val_mae: 0.0064 - val_mse: 2.2013e-04 - val_log_mse: -8.7649 - val_log_mae: -5.0860 - val_exp_mse_mag: -3.5682 - val_exp_mae_mag: -2.1199 - val_exp_R2: 0.9849 - val_R2: 0.9848
Model: "linear_embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 species_input (InputLayer)  [(None, 53)]              0         
                                                                 
 linear_embedding (Dense)    (None, 4)                 216       
                                                                 
Total params: 216
Trainable params: 216
Non-trainable params: 0
_________________________________________________________________
                MAE           MSE 

Epoch 2/5
129/129 [==============================] - 2s 12ms/step - loss: 0.0204 - mae: 0.0182 - mse: 0.0020 - log_mse: -6.5016 - log_mae: -4.0565 - exp_mse_mag: -2.4896 - exp_mae_mag: -1.6435 - exp_R2: 0.8517 - R2: 0.8659 - val_loss: 0.0157 - val_mae: 0.0140 - val_mse: 0.0017 - val_log_mse: -6.8063 - val_log_mae: -4.3240 - val_exp_mse_mag: -2.6196 - val_exp_mae_mag: -1.7532 - val_exp_R2: 0.8822 - val_R2: 0.8961
Epoch 3/5
129/129 [==============================] - 2s 12ms/step - loss: 0.0155 - mae: 0.0137 - mse: 0.0014 - log_mse: -7.0411 - log_mae: -4.3612 - exp_mse_mag: -2.7101 - exp_mae_mag: -1.7719 - exp_R2: 0.9003 - R2: 0.9144 - val_loss: 0.0142 - val_mae: 0.0112 - val_mse: 0.0012 - val_log_mse: -7.1574 - val_log_mae: -4.5470 - val_exp_mse_mag: -2.7611 - val_exp_mae_mag: -1.8427 - val_exp_R2: 0.9122 - val_R2: 0.9220
Epoch 4/5
129/129 [==============================] - 2s 12ms/step - loss: 0.0137 - mae: 0.0115 - mse: 0.0011 - log_mse: -7.3268 - log_mae: -4.5260 - exp_mse_mag: -2.828

KeyboardInterrupt: 

In [ ]:
""" prepare SimpleDNN for loading (from prior experiments) """

exprExec = DNNExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(SimpleDNNModelFactory())

In [ ]:
""" prepare PCDNNV1 for loading (from prior experiments) """

exprExec = PCDNNV1ExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(PCDNNV1ModelFactory())

In [3]:
""" prepare PCDNNV2 for loading (from prior experiments) """

exprExec = PCDNNV2ExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(PCDNNV2ModelFactory())

Parent DNNModelFactory Instantiated
Parent DNNModelFactory Instantiated


/home/dwyerdei/rom_project2/src/experiment_executor/pcdnn_v2_experiment_executor.py:48: UserWarning: manually overriding n models to 1! change this!
  warnings.warn('manually overriding n models to 1! change this!')


## Rapid Model Testing: 
### (requires setting up PCDNNV2 for loading)

In [14]:
# fix seeds
import random
import numpy as np
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

dataType = 'randomequaltraintestsplit' #'frameworkincludedtrainexcludedtest'
inputType = 'AllSpeciesAndZmix'
dataSetMethod = f'{inputType}_{dataType}'
opscaler = "MinMaxScaler"# 'PositiveLogNormal'
ipscaler = "MinMaxScaler"# 'PositiveLogNormal'
ZmixPresent = 'Y'
concatenateZmix = 'Y'
kernel_constraint = 'Y'
kernel_regularizer = 'Y'
activity_regularizer = 'Y'
noOfCpv = 4
noOfNeurons = 53

exprExec.modelFactory.loss='mse'
exprExec.modelFactory.activation_func='relu'
exprExec.modelFactory.dropout_rate=0.25
exprExec.modelFactory.skip_layers = False
exprExec.modelFactory.batch_norm = False
exprExec.modelFactory.basic = False
exprExec.debug_mode = False
exprExec.epochs_override = 30
exprExec.batch_size = 32
exprExec.n_models_override = 1

# initialize experiment executor...
exprExec.dm = dm
exprExec.df_experimentTracker = pd.DataFrame()
exprExec.modelType = 'PCDNNV2'

history = exprExec.executeSingleExperiment(noOfNeurons,dataSetMethod,dataType,inputType,ZmixPresent=ZmixPresent,
                                           noOfCpv=noOfCpv,concatenateZmix=concatenateZmix,kernel_constraint=kernel_constraint,
                                           kernel_regularizer=kernel_regularizer,activity_regularizer=activity_regularizer,
                                           opscaler=opscaler, ipscaler=ipscaler)

--------------------self.build_and_compile_pcdnn_v2_model----------------------
53 4 Y Y Y Y
Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 species_input (InputLayer)     [(None, 53)]         0           []                               
                                                                                                  
 zmix (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 linear_embedding (Functional)  (None, 4)            216         ['species_input[0][0]']          
                                                                                                  
 concatenated_zmix_linear_embed  (None, 5)           0           ['zmix[0][0]',                  

257/257 [==============================] - 5s 16ms/step - loss: 0.0375 - mae: 0.0552 - mse: 0.0125 - log_mse: -4.7181 - log_mae: -2.9616 - exp_mse_mag: -1.7531 - exp_mae_mag: -1.1897 - exp_R2: -0.0660 - R2: 0.1588 - val_loss: 0.0105 - val_mae: 0.0416 - val_mse: 0.0055 - val_log_mse: -5.5122 - val_log_mae: -3.2148 - val_exp_mse_mag: -2.1019 - val_exp_mae_mag: -1.3074 - val_exp_R2: 0.5898 - val_R2: 0.6271
Epoch 2/30
257/257 [==============================] - 4s 15ms/step - loss: 0.0118 - mae: 0.0433 - mse: 0.0075 - log_mse: -5.1998 - log_mae: -3.1835 - exp_mse_mag: -1.9436 - exp_mae_mag: -1.2810 - exp_R2: 0.3362 - R2: 0.4705 - val_loss: 0.0060 - val_mae: 0.0307 - val_mse: 0.0026 - val_log_mse: -6.1095 - val_log_mae: -3.5090 - val_exp_mse_mag: -2.3747 - val_exp_mae_mag: -1.4377 - val_exp_R2: 0.7601 - val_R2: 0.7745
Epoch 3/30
257/257 [==============================] - 4s 15ms/step - loss: 0.0088 - mae: 0.0379 - mse: 0.0057 - log_mse: -5.4896 - log_mae: -3.3191 - exp_mse_mag: -2.0717 - exp

Epoch 21/30
257/257 [==============================] - 4s 15ms/step - loss: 0.0035 - mae: 0.0226 - mse: 0.0019 - log_mse: -6.5893 - log_mae: -3.8286 - exp_mse_mag: -2.5377 - exp_mae_mag: -1.5607 - exp_R2: 0.8199 - R2: 0.8565 - val_loss: 0.0022 - val_mae: 0.0141 - val_mse: 5.6390e-04 - val_log_mse: -7.7924 - val_log_mae: -4.2881 - val_exp_mse_mag: -3.0954 - val_exp_mae_mag: -1.7767 - val_exp_R2: 0.9484 - val_R2: 0.9527
Epoch 22/30
257/257 [==============================] - 4s 15ms/step - loss: 0.0037 - mae: 0.0221 - mse: 0.0017 - log_mse: -6.6615 - log_mae: -3.8526 - exp_mse_mag: -2.5832 - exp_mae_mag: -1.5759 - exp_R2: 0.7992 - R2: 0.8421 - val_loss: 0.0035 - val_mae: 0.0161 - val_mse: 9.9109e-04 - val_log_mse: -7.4218 - val_log_mae: -4.1731 - val_exp_mse_mag: -2.8928 - val_exp_mae_mag: -1.7121 - val_exp_R2: 0.9249 - val_R2: 0.9402
Epoch 23/30
257/257 [==============================] - 4s 15ms/step - loss: 0.0035 - mae: 0.0218 - mse: 0.0018 - log_mse: -6.6897 - log_mae: -3.8705 - exp_m

In [ ]:
"""
import os, pickle
os.system('mkdir base_code_model')
with open('base_code_model/custom_objects.pickle', 'wb') as f:
        pickle.dump(exprExec.modelFactory.concreteClassCustomObject, f)

for i in range(200):
    model = exprExec.modelFactory.rebuild_model()
    model.save(f'base_code_model/base_code_model{i}.h5')
    
"""

## Results Plotting & Analysis

In [ ]:
loss = 727684307.3417714 # Update Me!

def print_scientific_notation(number):
    power = int(np.log(number)/np.log(10))
    print(f"Scientific Notation: {(loss/10**power)}*10^{power}")
print_scientific_notation(loss)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.plot(np.maximum(history.history['val_R2'][30:], 0))
plt.title('val_R^2 vs epochs')
plt.ylabel('percent')
plt.xlabel('epochs')
plt.show()

In [ ]:
import model_analyzer.model_analysis as model_analysis
import importlib; importlib.reload(model_analysis)

bestModel, experimentSettings = exprExec.modelFactory.openBestModel()
dm.createTrainTestData(experimentSettings['dataSetMethod'],experimentSettings['noOfCpv'], experimentSettings['ipscaler'], experimentSettings['opscaler'])

print(f'\nexperimentSettings: {experimentSettings}')
print(f'\nbestModel.input_shape: {bestModel.input_shape}')
inspector = model_analysis.ModelInspector(exprExec.modelFactory, dm)

In [ ]:
n_repeats = 5 if debug_mode else 20
inspector.plot_permutation_feature_importance(n_repeats=n_repeats)

In [ ]:
inspector.plot_partial_dependence()

### Print & Record Linear Embeddings Output (NOTE: Deprecated)

In [ ]:
import numpy as np

def inspect_PCA(X):
    PCA_dict = {}
    PCA_dict['X'] = X
    PCA_dict['Var'] = PCA_dict['X'].var(axis=0)
    
    # sort by explained variance
    sort_idx = np.argsort(PCA_dict['Var'])[::-1]
    PCA_dict['X'] = PCA_dict['X'][:, sort_idx]
    PCA_dict['Var'] = PCA_dict['Var'][sort_idx]
    PCA_dict['CumVar'] = np.add.accumulate(PCA_dict['Var'])
    return PCA_dict


X, Y, rom, zmix = dm.getAllData()
FullDataset = inspect_PCA(X)
try:    
    linearAutoEncoder = exprExec.modelFactory.getLinearEncoder()
    X = linearAutoEncoder.predict(FullDataset['X'])
    DNN_PCA = inspect_PCA(X)
except KeyError:
    print('No Linear Auto Encoder!')

In [ ]:
#dm.createDataset
import matplotlib.pyplot as plt
import copy
dm_PurePCA = copy.deepcopy(dm)
dm_PurePCA.createTrainTestData(dataSetMethod='PurePCA_randomequalflamesplit',
                               numCpvComponents=2, ipscaler=None, opscaler=None)

X,Y,rom,zmix = dm_PurePCA.getAllData()
PurePCA = inspect_PCA(X)
print(PurePCA['Var'])
print(DNN_PCA['Var'])

plt.plot(PurePCA['CumVar']/FullDataset['Var'].sum(), color='r')
plt.plot(DNN_PCA['CumVar']/FullDataset['Var'].sum(), color='b')
plt.title('PCA Total Variance Comparison')